##### Prototype — With Generative AI Tools

##### Importing the required libraries 

In [1]:
# (Generated with ChatGPT prompt: "Write the imports for a tabular ML regression pipeline with scikit-learn, XGBoost fallback, and visuals.")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Optional: XGBoost (GenAI suggested trying a boosted tree)
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False


##### Loading the dataset and preprocessing the data (GenAI-drafted utilities)

In [3]:
# (Generated with ChatGPT prompt: "Write a robust loader that parses datetime columns and engineers key OR features.")
def load_and_engineer(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)

    # Parse datetimes
    for c in ["Start Time", "End Time", "Wheels In", "Wheels Out"]:
        df[c] = pd.to_datetime(df[c], errors="coerce")

    # Core features
    df["Actual OR Duration (min)"] = (df["End Time"] - df["Start Time"]).dt.total_seconds() / 60
    df["Setup Duration (min)"]     = (df["Start Time"] - df["Wheels In"]).dt.total_seconds() / 60
    df["Room Occupancy (min)"]     = (df["Wheels Out"] - df["Wheels In"]).dt.total_seconds() / 60
    df["Start Hour"]               = df["Start Time"].dt.hour
    df["Day of Week"]              = df["Start Time"].dt.day_name()

    # Minimal cleaning for the prototype
    keep = ["Service", "CPT Description", "Booked Time (min)", "OR Suite",
            "Start Hour", "Day of Week", "Actual OR Duration (min)"]
    df = df[keep].dropna()
    df = df[df["Actual OR Duration (min)"] > 0]

    return df

df_ai = load_and_engineer("data/2022_Q1_OR_Utilization.csv")
df_ai.head()


C:\Users\User\AppData\Local\Temp\ipykernel_4984\640148119.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")
C:\Users\User\AppData\Local\Temp\ipykernel_4984\640148119.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")
C:\Users\User\AppData\Local\Temp\ipykernel_4984\640148119.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")
C:\Users\User\AppData\Local\Temp\ipykernel_4984\640148119.py:7: UserWarning: Could not infer format, so each 

,Service,CPT Description,Booked Time (min),OR Suite,Start Hour,Day of Week,Actual OR Duration (min)
0,Podiatry,"Partial ostectomy, fifth metatarsal head",90,1,7,Monday,93.0
1,Podiatry,"Neurectomy, intrinsic musculature of foot",60,1,10,Monday,48.0
2,Podiatry,Lapidus bunionectomy,150,1,12,Monday,22.0
3,Podiatry,Bunionectomy with distal osteotomy,120,1,13,Monday,57.0
4,Orthopedics,"Arthroplasty, knee, hinge prothesis",120,2,7,Monday,108.0
